In [8]:
import calendar_utils
import json

In [91]:
import importlib 
importlib.reload(calendar_utils)

<module 'calendar_utils' from '/Users/hayley/Documents/conv-ai/calendar_utils.py'>

In [39]:
from dotenv import load_dotenv
import os
import re
from datetime import datetime

load_dotenv()

True

In [72]:
# Basic ChatGPT class
import openai
from openai import OpenAI


class BasicChatGPT:
  def __init__(self, openai_api_key, max_tokens=100):
    openai.api_key = openai_api_key
    self.max_tokens = max_tokens
    self.messages = []
    self.client = OpenAI()

  def call(self):
    response = self.client.chat.completions.create(
      model="gpt-4-1106-preview",
      # model="gpt-3.5-turbo",
      response_format={ "type": "json_object" },
      messages=self.messages,
      max_tokens=self.max_tokens
    )
    return response

  def prompt(self, text):
    # Update the messages so far
    self.messages.append({
        "role": "user",
        "content": text,
    })

    # Call ChatGPT
    response = self.call()

    # Save the returned message
    message = response["choices"][0]["message"]
    self.messages.append(message)

    return message["content"]


In [23]:
# # Run an interactive console to chat with ChatGPT
# def run_console(chatgpt):
#     while True:
#         # Receive user input
#         prompt = input("User: ")

#         # Exit the loop if the user entered "exit"
#         if prompt == "exit":
#             break

#         # Receive and display ChatGPT response
#         response = chatgpt.prompt(prompt)
#         print("ChatGPT:", response)
#     print("The chat has ended")

In [73]:
# Make a ChatGPT object
basic_chatgpt = BasicChatGPT(os.environ.get("OPENAI_API_KEY"), max_tokens = 120)

# run_console(basic_chatgpt)

In [9]:
# calendar service 정의하는 부분 -> credentials.json 이 경로에 있어야 하고 
# 그리고 팝업 뜨면 너가 authorize 해줘야함 그리고 너 노트북이나 데탑 로컬에서 해야 함
# https://developers.google.com/resources/api-libraries/documentation/calendar/v3/python/latest/calendar_v3.events.html#list
service = calendar_utils.get_calendar_service()

In [ ]:
## if user asks about "today's schedule"
## first detect today
## then find what's today
## then request calendar for today's schedule

In [75]:
# first detect today and then find what date is today

input_text = f""" Today is {datetime.datetime.today().date()}.
Detect any time-related phrase from the given input from the user and resolve it into a date in the format of YYYY/MM/DD.
Examples include today, tomorrow, this Wednesday, next Tuesday, last Friday, 11/13, November 5th, 13th of July.
When the detected word is day of week, make sure to include adjective in front of it such as "this", "next", "upcoming", "last", "past".
\n\n
Your output must be in this format. Json("detected_phrase": <detected phrase>, "date": <YYYY/MM/DD>)

\n\n
Example 1
Input: Today is 2023/1/2. ... I want to schedule a meeting at 5PM tomorrow.
Output: tomorrow, 2023/1/3

Example 2
Input: Today is 2023/1/2. ... What time does the class start next Friday?
Output: next Friday, 2023/1/13

Example 3
Input: Today is 2023/1/2. ... What is the date of this upcoming Friday?
Output: this upcoming Friday, 2023/1/6
\n\nInput: 
"""

input_text += "I want to see a list of schedule from Monday two weeks later."



basic_chatgpt.messages.append({
    "role": "user",
    "content": input_text,
})

# Call ChatGPT
response = basic_chatgpt.call()
message = response.choices[0].message.content
init_result = json.loads(message)

print(message)
# Save the returned message
# message = response["choices"][0]["message"]
# self.messages.append(message)



{"detected_phrase": "Monday two weeks later", "date": "2023/12/04"}


In [102]:
# mydate = datetime.datetime.strptime(init_result['date'], "%Y/%m/%d").isoformat() + "Z"
# mydate_p1 = (datetime.datetime.strptime(init_result['date'], "%Y/%m/%d") + datetime.timedelta(days=1)).isoformat() + "Z"
mydate = datetime.datetime.now(datetime.UTC).isoformat().split('+')[0] + "Z"
mydate_p1 = (datetime.datetime.now(datetime.UTC) + datetime.timedelta(days=1)).isoformat().split('+')[0] + "Z"

In [104]:
#https://developers.google.com/resources/api-libraries/documentation/calendar/v3/python/latest/calendar_v3.events.html#list

calendar_utils.get_event_list_recent(timeMin=mydate, timeMax=mydate_p1)

2023-10-08T23:00:00+09:00 기도
2023-11-22 규인 휴가


[{'kind': 'calendar#event',
  'etag': '"3393470160096000"',
  'id': '_8os38chm650j2ba274qj2b9k70rj2b9p6cr48ba48p2j8da16spj6ci460',
  'status': 'confirmed',
  'htmlLink': 'https://www.google.com/calendar/event?eid=XzhvczM4Y2htNjUwajJiYTI3NHFqMmI5azcwcmoyYjlwNmNyNDhiYTQ4cDJqOGRhMTZzcGo2Y2k0NjBfMjAyMzEwMDhUMTQwMDAwWiBoYXlsZXlzb245MkBt',
  'created': '2023-10-08T03:17:49.000Z',
  'updated': '2023-10-08T03:18:00.048Z',
  'summary': '기도',
  'creator': {'email': 'hayleyson92@gmail.com', 'self': True},
  'organizer': {'email': 'hayleyson92@gmail.com', 'self': True},
  'start': {'dateTime': '2023-10-08T23:00:00+09:00', 'timeZone': 'Asia/Seoul'},
  'end': {'dateTime': '2023-10-09T00:00:00+09:00', 'timeZone': 'Asia/Seoul'},
  'recurrence': ['RRULE:FREQ=DAILY'],
  'iCalUID': 'F84261A1-B951-4871-936D-DFE45A7332D0',
  'sequence': 1,
  'reminders': {'useDefault': True},
  'eventType': 'default'},
 {'kind': 'calendar#event',
  'etag': '"3399772430664000"',
  'id': '_68pj6d2660pj4ba460p3ab9k6so3eba18gr

In [49]:
# 이거 내가 룰로 짜서 this monday -> 2023/11/27 이렇게 파싱하려고 했던 흔적임. 근데 그냥 chatgpt gpt4 한테 물어봤는데 잘해서 그냥 이거로 안했음..
# 그렇게 할 때의 input_text 는 이거였음 아래 참고

# input_text = """
# Detect any time-related word from the given input from the user.
# Examples include today, tomorrow, this Wednesday, next Tuesday, last Friday, 11/13, November 5th, 13th of July.
# When the detected word is day of week, make sure to include adjective in front of it such as "this", "next", "upcoming", "last", "past".
# \n\n
# Example 1
# Input: I want to schedule a meeting at 5PM tomorrow.
# Output: tomorrow

# Example 2
# Input: What time does the class start next Friday?
# Output: next Friday

# Example 3
# Input: What is the date of this upcoming Friday?
# Output: this upcoming Friday
# \n\nInput: 
# """


# query_date = None
# found_time_related_phrase = 'this Monday'
# found_time_related_phrase = found_time_related_phrase.lower()
# if found_time_related_phrase == 'today':
#     query_date = datetime.datetime.today().date()
# elif found_time_related_phrase == 'tomorrow':
#     query_date = datetime.datetime.today().date() + datetime.timedelta(days=1)
# elif found_time_related_phrase == 'yesterday':
#     query_date = datetime.datetime.today().date() - datetime.timedelta(days=1)
# elif 'day' in found_time_related_phrase:
#     phrases = found_time_related_phrase.split()
#     prefix = phrases[0]
#     dow = phrases[-1]
#     dow2num={'monday':0, 'tuesday':1, 'wednesday':2, 'thursday':3, 'friday': 4, 'saturday':5, 'sunday':6}
#     today = datetime.date.today()
#     this_dow = today + datetime.timedelta( (dow2num[dow]-today.weekday()) % 7 )
#     if prefix == 'this': # dow closest from today in the future
#         query_date = this_dow
#     elif prefix == 'next': # dow one week from this dow
#         query_date = this_dow + datetime.timedelta(7)
#     elif prefix == 'last': # dow one week prior to this dow
#         query_date = this_dow - datetime.timedelta(7)
# print(query_date)

2023-11-27
